# Churn case. Part 2. Analysis of LTV

Lifetime value (LTV) is the total worth to a business of a customer over the while period of their relationship. It's an important metric as it costs less to keep existings customers than to acquire new ones, so the idea of increasing the value of your existig customers is a great way to drive growth. 

## 1. For those customers who unsubscribed the service, what was their average LTV? and how long did they usually stay in the service?

In [4]:
# importing files and creating connection

In [5]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
conn= mysql.connector.connect(host='localhost',user='root',password='#',db='churn')

engine = create_engine('mysql+mysqlconnector://root:#@localhost/churn', echo=False) 
cur = conn.cursor()

In [6]:
churn_all=churn_df = pd.read_sql("SELECT * FROM churn_all" , conn)

C:\Users\THE_DE_V_IL\AppData\Local\Temp\ipykernel_9560\4095631495.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  churn_all=churn_df = pd.read_sql("SELECT * FROM churn_all" , conn)


In [7]:
## Extract those who unsubscribed the serviced (Churn = Yes)

churn_df = pd.read_sql("SELECT * FROM churn_all WHERE Churn= 'Yes'", conn)

C:\Users\THE_DE_V_IL\AppData\Local\Temp\ipykernel_9560\3147114677.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  churn_df = pd.read_sql("SELECT * FROM churn_all WHERE Churn= 'Yes'", conn)


In [8]:
churn_df.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,No,Yes,Yes,8,Month-to-month,Electronic check,Yes,99.65,820.50,Yes
3,7892-POOKP,Female,No,Yes,Yes,California,34.062125,-118.315709,90010,Yes,...,Yes,Yes,Yes,28,Month-to-month,Electronic check,Yes,104.80,3046.05,Yes
4,0280-XJGEX,Male,No,No,Yes,California,34.039224,-118.266293,90015,Yes,...,No,Yes,Yes,49,Month-to-month,Bank transfer (automatic),Yes,103.70,5036.30,Yes


In [9]:
# Examine the distribution of TotalCharges
churn_df["TotalCharges"].describe()

count    1869.000000
mean     1531.796094
std      1890.822994
min        18.850000
25%       134.500000
50%       703.550000
75%      2331.300000
max      8684.800000
Name: TotalCharges, dtype: float64

We can see that around 20% of the TotalCharges are very high, so let's divide the data to see each distribution.

In [15]:
churn_df.TotalCharges.quantile(0.9)

4556.790000000002

In [10]:
# Find the 80th percentile of the data in TotalCharges
churn_df.TotalCharges.quantile(0.8)

2840.4100000000003

In [ ]:
# Divide the data by the 80th percentile of the data from the TotalCharges variable
total_charges_under80 = churn_df.query("TotalCharges <= 2840.41")
total_charges_above80 = churn_df.query("TotalCharges > 2840.41")

In [ ]:
# Show the distribution of people under the 80th percentile
total_charges_under80.TotalCharges.describe()

count    1495.000000
mean      711.265819
std       766.848197
min        18.850000
25%        85.025000
50%       371.650000
75%      1128.225000
max      2839.650000
Name: TotalCharges, dtype: float64

In [ ]:
# Show the distribution of people above the 80th percentile
total_charges_above80.TotalCharges.describe()

count     374.000000
mean     4811.723262
std      1436.724288
min      2841.550000
25%      3522.462500
50%      4571.100000
75%      5891.212500
max      8684.800000
Name: TotalCharges, dtype: float64

In [ ]:
# Show the distribution of the tenure of people under the 80th percentile
total_charges_under80.Tenure.describe()

count    1495.000000
mean        9.935117
std        10.742349
min         1.000000
25%         1.000000
50%         6.000000
75%        15.000000
max        61.000000
Name: Tenure, dtype: float64

In [ ]:
# Show the distribution of the tenure of people above the 80th percentile
total_charges_above80.Tenure.describe()

count    374.000000
mean      50.133690
std       12.334841
min       27.000000
25%       40.000000
50%       49.500000
75%       60.000000
max       72.000000
Name: Tenure, dtype: float64

The average LTV of the 80% of those who unsubscribed is \$ 711, and their tenure is near 10 months. On the hand, the average LTV of the 20 % of those who unsubscribed is $ 4811 which is huge , and their tenure is near 50 months.  

## 2. What kinds of services they subscribed when they were still a customer?

### phone service

In [ ]:
# under 80
pd.read_sql("select count(1) from churn_all where TotalCharges <= 2840.41 and Churn = 'Yes' group by PhoneService", conn)

C:\Users\THE_DE_V_IL\AppData\Local\Temp\ipykernel_14412\169793409.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("select count(1) from churn_all where TotalCharges <= 2840.41 and Churn = 'Yes' group by PhoneService", conn)


,count(1)
0,1334
1,161


In [ ]:
churn_all["Gender"].value_counts() # example

Male      3555
Female    3488
Name: Gender, dtype: int64

In [ ]:
# under 80
total_charges_under80.PhoneService.value_counts()

Yes    1334
No      161
Name: PhoneService, dtype: int64

In [ ]:
# Example of shape
churn_all.shape

(7043, 25)

In [ ]:
# Under 80 (percentage)
total_charges_under80.PhoneService.value_counts() / total_charges_under80.shape[0]

Yes    0.892308
No     0.107692
Name: PhoneService, dtype: float64

In [ ]:
# Above 80 (percentage)
total_charges_above80.PhoneService.value_counts() / total_charges_above80.shape[0]

Yes    0.975936
No     0.024064
Name: PhoneService, dtype: float64

For the top 20% (high LTV) only 2% of them didn't use the phone service. On the other hand, lower 80% (LTV) had around 11% of the people that didn't use the phone service. That is around 5 times more as the top 20%. Apparently if you are not subscribed to the phone service you are more likely to be in the bottom 80% of the LTV. We as a company can try to sell this service to more people.

### MutipleLines

In [ ]:
total_charges_under80_use_phone = total_charges_under80.query("PhoneService == 'Yes'")

total_charges_under80_use_phone.MultipleLines.value_counts() / total_charges_under80_use_phone.shape[0]

No     0.590705
Yes    0.409295
Name: MultipleLines, dtype: float64

In [ ]:
total_charges_above80_use_phone = total_charges_above80.query("PhoneService == 'Yes'")

total_charges_above80_use_phone.MultipleLines.value_counts() / total_charges_above80_use_phone.shape[0]

Yes    0.832877
No     0.167123
Name: MultipleLines, dtype: float64

For the top 20% of LTV who used the phone service 84% of them used multiple lines, which is 2 times as much as the proportion of people in the low 80% LTV that is around 40%. Having multiple lines is going to get you closer to be in the top 20% of the LTV.

### InternetService

In [ ]:
# Under 80 (percentage)
total_charges_under80.InternetService.value_counts() / total_charges_under80.shape[0]

Fiber optic    0.640803
DSL            0.283612
No             0.075585
Name: InternetService, dtype: float64

In [ ]:
# Above 80 (percentage)
total_charges_above80.InternetService.value_counts() / total_charges_above80.shape[0]

Fiber optic    0.906417
DSL            0.093583
Name: InternetService, dtype: float64

All the people in the high 20% LTV used internet service. On the other hand 8% of the people in the low 80th percentile of the LTV did not have internet connection. Also, in the top 20% (high LTV) 90% of them had fiber optic, in comparison to only 64% in the low LTV. So as a company if we want to increase the LTV of customers, we should recommend using internet and Fiber Optic connection.

### Other services

In [ ]:
import numpy as np

In [ ]:
total_charges_under80_use_internet = total_charges_under80.query('InternetService!="No"')
proportion_internet_sub_service_under80 = np.array([total_charges_under80_use_internet.query('OnlineSecurity=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('TechSupport=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('OnlineBackup=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('DeviceProtection=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('StreamingTV=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('StreamingMovies=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0]])

In [ ]:
proportion_internet_sub_service_under80

array([0.1316932 , 0.13603473, 0.22720695, 0.23516643, 0.37698987,
       0.3813314 ])

In [ ]:
total_charges_above80_use_internet = total_charges_above80.query('InternetService!="No"')
proportion_internet_sub_service_above80 = np.array([total_charges_above80_use_internet.query('OnlineSecurity=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('TechSupport=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('OnlineBackup=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('DeviceProtection=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('StreamingTV=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('StreamingMovies=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0]])

In [ ]:
proportion_internet_sub_service_above80

array([0.30213904, 0.32620321, 0.55882353, 0.58823529, 0.78342246,
       0.77807487])

In the top 20% LTV, both streaming movies and streaming tv percentages are around 80%, in comparison in the low 80% LTV they are around 40%. Device protection and online backup for the top 20% LTV are around 55%, in comparison to only around 22% in the low LTV. As a company we would like to sell to customers streaming TV and movies and also online backup and device protection if we want to increase LTV.

## 3. For those who churned what is the proportion of each kind of contract?

In [ ]:
# Under 80 (percentage)
total_charges_under80.Contract.value_counts() / total_charges_under80.shape[0]

Month-to-month    0.959197
One year          0.036789
Two year          0.004013
Name: Contract, dtype: float64

In [ ]:
# Above 80 (percentage)
total_charges_above80.Contract.value_counts() / total_charges_above80.shape[0]

Month-to-month    0.590909
One year          0.296791
Two year          0.112299
Name: Contract, dtype: float64

## 4. For those who did not churn what is the proportion of each kind of contract?

In [ ]:
## Extract those who unsubscribed the services (Churn = Yes)

paying_df = pd.read_sql("select * from churn_all where Churn = 'No'", conn)

C:\Users\THE_DE_V_IL\AppData\Local\Temp\ipykernel_14412\167045665.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  paying_df = pd.read_sql("select * from churn_all where Churn = 'No'", conn)


In [ ]:
paying_df.TotalCharges.describe()

count    5174.000000
mean     2552.882494
std      2327.590132
min        18.800000
25%       579.575000
50%      1679.525000
75%      4262.850000
max      8672.450000
Name: TotalCharges, dtype: float64

In [ ]:
# Find the 80th percentile of the data in TotalCharges
paying_df.TotalCharges.quantile(0.8)

4890.900000000001

In [ ]:
# Divide the data by the 80th percentile of the data from the TotalCharges variable
paying_total_charges_under80 = paying_df.query("TotalCharges <= 4895.8")
paying_total_charges_above80 = paying_df.query("TotalCharges > 4895.8")

In [ ]:
# Under 80 (percentage)
paying_total_charges_under80.Contract.value_counts() / paying_total_charges_under80.shape[0]

Month-to-month    0.499396
Two year          0.255011
One year          0.245593
Name: Contract, dtype: float64

In [ ]:
# Above 80 (percentage)
paying_total_charges_above80.Contract.value_counts() / paying_total_charges_above80.shape[0]

Two year          0.572120
One year          0.280736
Month-to-month    0.147144
Name: Contract, dtype: float64

- If we want to stop Churning we would like to sell contracts to people for at least one year, and much better for 2 years (Conjecture).
- We have 57% of the people that stayed in the company in the top 20% LTV with 2 year contracts, and only 11% of the people in the top 20% had a 2 year contract and left the company.
- 95% of the people that left the company and are in the low 80% have a month-to-month contract, in contrast to only 50% of the people in the low 80% that stayed in the company. We can say that having a month-to-month contract is one of the reasons that people are leaving the company (conjecture).